# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [1]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [2]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googlebot"/>\n  <meta co

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [3]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [4]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [5]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [6]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [7]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [8]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [9]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'11.1-21.5', u'MPG': u'35.6', u'FTM-FTA': u'6.4-6.9', u'FG%': u'.515', u'GP': u'14', u'PLAYER': u'Stephen Curry', u'FT%': u'.938', u'TEAM': u'GS', u'3PM-3PA': u'5.1-11.5', 'POSITION': u'PG', u'PTS': u'33.6', u'3P%': u'.441'}, {u'FGM-FGA': u'9.8-21.5', u'MPG': u'35.2', u'FTM-FTA': u'6.9-8.5', u'FG%': u'.459', u'GP': u'13', u'PLAYER': u'Russell Westbrook', u'FT%': u'.811', u'TEAM': u'OKC', u'3PM-3PA': u'1.9-5.8', 'POSITION': u'PG', u'PTS': u'28.5', u'3P%': u'.329'}, {u'FGM-FGA': u'7.8-20.5', u'MPG': u'39.0', u'FTM-FTA': u'10.2-11.6', u'FG%': u'.378', u'GP': u'13', u'PLAYER': u'James Harden', u'FT%': u'.881', u'TEAM': u'HOU', u'3PM-3PA': u'2.5-9.6', 'POSITION': u'SG', u'PTS': u'28.2', u'3P%': u'.256'}, {u'FGM-FGA': u'9.6-19.1', u'MPG': u'34.8', u'FTM-FTA': u'6.3-7.0', u'FG%': u'.503', u'GP': u'8', u'PLAYER': u'Kevin Durant', u'FT%': u'.893', u'TEAM': u'OKC', u'3PM-3PA': u'2.6-5.9', 'POSITION': u'SF', u'PTS': u'28.1', u'3P%': u'.447'}, {u'FGM-FGA': u'8.6-20.0', u'MPG': u'32.

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [10]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.441,5.1-11.5,.515,11.1-21.5,.938,6.4-6.9,14,35.6,Stephen Curry,PG,33.6,GS
1,.329,1.9-5.8,.459,9.8-21.5,.811,6.9-8.5,13,35.2,Russell Westbrook,PG,28.5,OKC
2,.256,2.5-9.6,.378,7.8-20.5,.881,10.2-11.6,13,39.0,James Harden,SG,28.2,HOU
3,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,28.1,OKC
4,.412,1.8-4.3,.431,8.6-20.0,.764,8.5-11.1,8,32.3,DeMarcus Cousins,PF,27.5,SAC


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [11]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,14,35.6,33.6,11.1-21.5,.515,5.1-11.5,.441,6.4-6.9,.938
1,Russell Westbrook,PG,OKC,13,35.2,28.5,9.8-21.5,.459,1.9-5.8,.329,6.9-8.5,.811
2,James Harden,SG,HOU,13,39.0,28.2,7.8-20.5,.378,2.5-9.6,.256,10.2-11.6,.881
3,Kevin Durant,SF,OKC,8,34.8,28.1,9.6-19.1,.503,2.6-5.9,.447,6.3-7.0,.893
4,DeMarcus Cousins,PF,SAC,8,32.3,27.5,8.6-20.0,.431,1.8-4.3,.412,8.5-11.1,.764


### Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [19]:
#Import BeautiFul soup (web scraping tool)
from bs4 import BeautifulSoup

#Import urllib module
import urllib2

url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false'

#Define a function get_cols that retrieves the names of the header columns given a table element 
#(skip the ranks, split the positions)

cols = []

def get_cols(url):
    #Use the urlopen and read() functions from the urllib module to open and read from a URL into an object? called page
    page = urllib2.urlopen(url).read()

    #create a BeautifulSoup object? called soup by running the BeutifulSoup function on the page we imported
    soup = BeautifulSoup(page)

    #This line confirms that the object we created belongs toe the BeautifulSoup class
    #print type(soup)

    #The prettify() method will turn a Beautiful Soup parse tree into a nicely formatted Unicode string, with each HTML/XML tag on its own line:
    #This shows us the whole webpage.
    soup.prettify()[0:1000]

    #But we only want the table. The find() method helps us find the table, and we prettify it too.
    table_div = soup.find(id='my-players-table')
    #print table_div.prettify()[0:4000]

    #I don't quite understand these extra steps to get the table. What's going on here?!?!
    table = table_div.find("table")
    #print table

    #Find the header row of the table by searching for tags under the table tag whose class attritbute is "colhead".
    table_head = table.find(attrs={"class":'colhead'})
    #print table_head.prettify()

    #Find the actual col values by searching for the 'td' tags, which is the tag for table data.
    header_cols = table_head.findAll('td')
    #print header_cols

    #Step through these columns and save them to a list.
    #Note that we don't want the 'RK' column, and we want to split the player column into separate columns for player and position.
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    print cols

get_cols(url)

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


In [20]:
#Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  
#You can use either the dict approach or the numpy array approach.

#Moved this outside the function so it doesn't reset each time we iterate through URLs.
players_stats_dicts = []

def get_data(url):

    #Use the urlopen and read() functions from the urllib module to open and read from a URL into an object? called page
    page = urllib2.urlopen(url).read()

    #create a BeautifulSoup object? called soup by running the BeutifulSoup function on the page we imported
    soup = BeautifulSoup(page)

    #This line confirms that the object we created belongs toe the BeautifulSoup class
    #print type(soup)

    #The prettify() method will turn a Beautiful Soup parse tree into a nicely formatted Unicode string, with each HTML/XML tag on its own line:
    #This shows us the whole webpage.
    soup.prettify()[0:1000]

    #But we only want the table. The find() method helps us find the table, and we prettify it too.
    table_div = soup.find(id='my-players-table')
    #print table_div.prettify()[0:4000]

    #I don't quite understand these extra steps to get the table. What's going on here?!?!
    table = table_div.find("table")
    #print table

    #The table rows are indicated by the tag 'tr'. 
    #Again we can find them all and iterate through them. Within each row we iterate through the respective columns.
    table_rows = table.findAll('tr')
    #print table_rows

    #Create a dictionary version of player stats with all of the data points that we want

    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        player_stats = dict(zip(cols, player_stats))
        players_stats_dicts.append(player_stats)

In [21]:
#Run through all URLs (40 players per page) and append data to dictionary using get_data function.
for x in range(1, 402):
    if x % 40 == 1:
        url_no = str(x)
        url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/' + url_no 
        get_data(url)


In [22]:
#See what the dictionary looks like.
print players_stats_dicts

[{u'FGM-FGA': u'11.1-21.5', u'MPG': u'35.6', u'FTM-FTA': u'6.4-6.9', u'FG%': u'.515', u'GP': u'14', u'PLAYER': u'Stephen Curry', u'FT%': u'.938', u'TEAM': u'GS', u'3PM-3PA': u'5.1-11.5', 'POSITION': u'PG', u'PTS': u'33.6', u'3P%': u'.441'}, {u'FGM-FGA': u'9.8-21.5', u'MPG': u'35.2', u'FTM-FTA': u'6.9-8.5', u'FG%': u'.459', u'GP': u'13', u'PLAYER': u'Russell Westbrook', u'FT%': u'.811', u'TEAM': u'OKC', u'3PM-3PA': u'1.9-5.8', 'POSITION': u'PG', u'PTS': u'28.5', u'3P%': u'.329'}, {u'FGM-FGA': u'9.6-19.1', u'MPG': u'34.8', u'FTM-FTA': u'6.3-7.0', u'FG%': u'.503', u'GP': u'8', u'PLAYER': u'Kevin Durant', u'FT%': u'.893', u'TEAM': u'OKC', u'3PM-3PA': u'2.6-5.9', 'POSITION': u'SF', u'PTS': u'28.1', u'3P%': u'.447'}, {u'FGM-FGA': u'7.8-20.5', u'MPG': u'38.9', u'FTM-FTA': u'9.9-11.1', u'FG%': u'.380', u'GP': u'14', u'PLAYER': u'James Harden', u'FT%': u'.885', u'TEAM': u'HOU', u'3PM-3PA': u'2.5-9.2', 'POSITION': u'SG', u'PTS': u'27.9', u'3P%': u'.271'}, {u'FGM-FGA': u'8.6-19.9', u'MPG': u'33.1

In [23]:
#Import the dictionary into a pandas dataframe.
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()


,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.441,5.1-11.5,.515,11.1-21.5,.938,6.4-6.9,14,35.6,Stephen Curry,PG,33.6,GS
1,.329,1.9-5.8,.459,9.8-21.5,.811,6.9-8.5,13,35.2,Russell Westbrook,PG,28.5,OKC
2,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,28.1,OKC
3,.271,2.5-9.2,.380,7.8-20.5,.885,9.9-11.1,14,38.9,James Harden,SG,27.9,HOU
4,.389,1.6-4.0,.430,8.6-19.9,.779,9.0-11.6,9,33.1,DeMarcus Cousins,PF,27.7,SAC


In [24]:
#Now repeat with the assists table, using the get_cols function.
cols = []
url = 'http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false'
get_cols(url)

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'AST', u'APG', u'TO', u'TOPG', u'AP48M', u'AST/TO']


In [25]:
#Run through all assists URLs (40 players per page) and append data to a dictionary using get_data function.
#Should I give this dictionary a different name or can I just clear the old one now that my other data is in df?
players_stats_dicts = []

for x in range(1, 402):
    if x % 40 == 1:
        url_no = str(x)
        url = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/' + url_no 
        get_data(url)

In [26]:
#See what the new dictionary looks like.
print players_stats_dicts

[{u'MPG': u'35.2', u'GP': u'13', u'AST': u'132', u'PLAYER': u'Russell Westbrook', u'TO': u'64', u'AP48M': u'13.9', u'TEAM': u'OKC', u'TOPG': u'4.9', 'POSITION': u'PG', u'APG': u'10.2', u'AST/TO': u'2.06'}, {u'MPG': u'36.0', u'GP': u'14', u'AST': u'142', u'PLAYER': u'Rajon Rondo', u'TO': u'54', u'AP48M': u'13.5', u'TEAM': u'SAC', u'TOPG': u'3.9', 'POSITION': u'PG', u'APG': u'10.1', u'AST/TO': u'2.63'}, {u'MPG': u'30.8', u'GP': u'9', u'AST': u'76', u'PLAYER': u'Ricky Rubio', u'TO': u'19', u'AP48M': u'13.2', u'TEAM': u'MIN', u'TOPG': u'2.1', 'POSITION': u'PG', u'APG': u'8.4', u'AST/TO': u'4.00'}, {u'MPG': u'32.7', u'GP': u'10', u'AST': u'84', u'PLAYER': u'John Wall', u'TO': u'41', u'AP48M': u'12.3', u'TEAM': u'WSH', u'TOPG': u'4.1', 'POSITION': u'PG', u'APG': u'8.4', u'AST/TO': u'2.05'}, {u'MPG': u'32.0', u'GP': u'9', u'AST': u'72', u'PLAYER': u'Chris Paul', u'TO': u'24', u'AP48M': u'12.0', u'TEAM': u'LAC', u'TOPG': u'2.7', 'POSITION': u'PG', u'APG': u'8.0', u'AST/TO': u'3.00'}, {u'MPG': 

In [27]:
#Create a second dataframe with assist stats
df2 = pd.DataFrame.from_dict(players_stats_dicts)
df2.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,TOPG
0,13.9,10.2,132,2.06,13,35.2,Russell Westbrook,PG,OKC,64,4.9
1,13.5,10.1,142,2.63,14,36.0,Rajon Rondo,PG,SAC,54,3.9
2,13.2,8.4,76,4.00,9,30.8,Ricky Rubio,PG,MIN,19,2.1
3,12.3,8.4,84,2.05,10,32.7,John Wall,PG,WSH,41,4.1
4,12.0,8.0,72,3.00,9,32.0,Chris Paul,PG,LAC,24,2.7


In [30]:
#Merge data frames
df3 = pd.DataFrame.merge(df, df2, on=['PLAYER', 'POSITION'], how='inner')
df3.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP_x,MPG_x,PLAYER,POSITION,...,TEAM_x,AP48M,APG,AST,AST/TO,GP_y,MPG_y,TEAM_y,TO,TOPG
0,.441,5.1-11.5,.515,11.1-21.5,.938,6.4-6.9,14,35.6,Stephen Curry,PG,...,GS,7.6,5.6,79,1.46,14,35.6,GS,54,3.9
1,.329,1.9-5.8,.459,9.8-21.5,.811,6.9-8.5,13,35.2,Russell Westbrook,PG,...,OKC,13.9,10.2,132,2.06,13,35.2,OKC,64,4.9
2,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,...,OKC,4.1,3.0,24,1.33,8,34.8,OKC,18,2.3
3,.271,2.5-9.2,.380,7.8-20.5,.885,9.9-11.1,14,38.9,James Harden,SG,...,HOU,7.9,6.4,90,1.34,14,38.9,HOU,67,4.8
4,.389,1.6-4.0,.430,8.6-19.9,.779,9.0-11.6,9,33.1,DeMarcus Cousins,PF,...,SAC,3.9,2.7,24,0.83,9,33.1,SAC,29,3.2
